In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif
from skbold.preproc import ConfoundRegressor, MajorityUndersampler
%matplotlib inline

/home/lukas/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ImportError: cannot import name 'ConfoundRegressor'

In [12]:
iters = 5
n_samp = 100
n_feat = 5
n_fold = 5
std = 1

perf = np.zeros(iters)
for i in range(iters):
    
    if i % int(iters / 5) == 0:
        print("Iteration %i" % i)

    # Generate data
    n_half = int(n_samp / 2)
    y = np.repeat([0, 1], repeats=n_half)
    #c = np.roll(y, 10)
    c = [0] * 10 + [1] * 40 + [0] * 10 + [1] * 40
    print(np.corrcoef(c, y))
    
    data = np.random.randn(n_samp, n_feat)
    #data[c == 1, :] += 5
    data[y == 1, :] += 1
    X = data
        
    skf = StratifiedKFold(n_splits=n_fold)
    accs = np.zeros(n_fold)
    # Loop over folds
    for ii, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svm', SVC(kernel='linear'))
        ])
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        pipeline.fit(X_train, y_train)
        accs[ii] = pipeline.score(X_test, y_test)
    perf[i] = accs.mean()
print(perf.mean())

Iteration 0
[[  1.00000000e+00   4.44089210e-17]
 [  4.44089210e-17   1.00000000e+00]]
Iteration 1
[[  1.00000000e+00   4.44089210e-17]
 [  4.44089210e-17   1.00000000e+00]]
Iteration 2
[[  1.00000000e+00   4.44089210e-17]
 [  4.44089210e-17   1.00000000e+00]]
Iteration 3
[[  1.00000000e+00   4.44089210e-17]
 [  4.44089210e-17   1.00000000e+00]]
Iteration 4
[[  1.00000000e+00   4.44089210e-17]
 [  4.44089210e-17   1.00000000e+00]]
0.846


In [213]:
y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
c = np.array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1])

X = np.random.randn(12, 5)
n_splits = 5

recode = {(0, 0): 0, (0, 1): 1, (1, 0): 2, (1, 1): 3}
z = [recode[(yi, ci)] for yi, ci in zip(y, c)]
    
success = 0
for i in range(10000):
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    cb = np.zeros(n_splits, dtype=bool)
    
    for ii, (train_idx, test_idx) in enumerate(skf.split(y=z, X=X)):
    
        this_c = c[train_idx]
        this_y = y[train_idx]
        
        if this_c[this_y == 0].mean() == this_c[this_y == 1].mean():
            #print("YES")
            cb[ii] = True
        
    if np.all(cb):
        success += 1

print(success)

ValueError: All the n_groups for individual classes are less than n_splits=5.